In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import threading
import time
import concurrent
import os, random
import csv
import glob

from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import STCIndicator

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
tickers = []
with open('symbols.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        tickers.append(row[0])
    tickers.pop(0) #remove zero index

In [ ]:

try:
    for symbol in tickers:
        try:
            df = pd.read_csv(f"OHLC1h/"+symbol+'1h.csv')
        except FileNotFoundError:
            pass
        
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')

        columns_dict = {'open': 'first', 
                        'high': 'max', 
                        'low': 'min', 
                        'close': 'last'}
        df = df.resample('D').agg(columns_dict)
        df = df.reset_index()
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['symbol'] = symbol


        #cpr calculation
        df['pivot'] = (df['high']+ df['low'] + df['close'])/3

        df['bc'] = (df['high'] + df['low'])/2
        df['tc'] = (df['pivot'] - df['bc']) + df['pivot']

        df['r1'] = 2*df['pivot'] - df['low']
        df['s1'] = 2*df['pivot'] - df['high']

        df['r2'] = df['pivot'] + (df['r1'] - df['s1'])
        df['s2'] = df['pivot'] - (df['r1'] - df['s1'])

        df['width'] = df['tc'] - df['bc']

        #extracting previous day high-low
        df['previous_high'] = df['high'].shift(1)
        df['previous_low'] = df['low'].shift(1)


        #checking for narrow cpr
        window = 9
        cpr_avg_width = df['width'].rolling(window = window).mean()
        cpr_sd = df['width'].rolling(window=window).std()

        df['narrow_cpr'] =df['width'] < cpr_avg_width - cpr_sd
        df['moderate_cpr'] = (df['width']>= cpr_avg_width-cpr_sd) & (df['width']<= cpr_avg_width+cpr_sd)
        df['wide_cpr'] =df['width'] > cpr_avg_width + cpr_sd

        df['width'] = df.apply(lambda x: 'narrow' if x.narrow_cpr == True 
                                else ('moderate' if x.moderate_cpr == True 
                                else 'wide'), axis =1)

        df = df.drop(['narrow_cpr','moderate_cpr','wide_cpr'], axis=1)


        #shift all rows after calculations
        df['pivot'] = df['pivot'].shift(1)
        df['bc'] = df['bc'].shift(1)
        df['tc'] = df['tc'].shift(1) 
        df['r1'] = df['r1'].shift(1)
        df['s1'] = df['s1'].shift(1)
        df['r2'] = df['r2'].shift(1)
        df['s2'] = df['s2'].shift(1)
        
        #checking for general trend
        df['pivot_shifted'] = df['pivot'].shift(1)
        df['cpr_trend'] = np.where(df['pivot']>df['pivot_shifted'], 'uptrend','downtrend')
        df.drop(['pivot_shifted'], axis=1)

        #check current day open 
        def check_trend_open(check_df):

            if (check_df['open'] > check_df['pivot'] and
                check_df['open'] > check_df['bc'] and
                check_df['open'] > check_df['tc']):
                return 'above'

            elif (check_df['open'] < check_df['pivot'] and
                  check_df['open'] < check_df['bc'] and
                  check_df['open'] < check_df['tc']):
                return 'below'

            else:
                return 'Neutral'

            return

        df['day_open'] = df.apply(check_trend_open, axis =1)

        #check for daily opportunity
        def check_trend_daily(check_df):

            if (check_df['day_open'] == 'above' and
                check_df['cpr_trend'] == 'uptrend'):
                return 'long'

            elif (check_df['day_open'] == 'below' and
                check_df['cpr_trend'] == 'downtrend'):
                return 'short'
            else:
                return 'unclear'

            return
        
        df['cpr_opportunity'] = df.apply(check_trend_daily, axis =1)
        
        column_to_move = df.pop('symbol')
        df.insert(0, 'symbol', column_to_move)


        df.to_csv(f"CPR/"+symbol+'cpr.csv', index= False)
except FileNotFoundError:
    pass

In [ ]:
file = glob.glob('CPR/*.csv')
random_files=np.random.choice(file,2)
file = file[1]
random_df =  pd.read_csv(file)
cpr_df = random_df.tail(1)

In [ ]:
for symbol in tickers:
    try:
        df = pd.read_csv(f"CPR/"+symbol+"cpr.csv")
        df = df.tail(1)
        cpr_df = cpr_df.append(df)
        cpr_df = cpr_df.drop_duplicates()
    except FileNotFoundError:
        pass

In [ ]:
cpr_df.to_csv('CPR/CPR.csv', index=False)

In [ ]:
pd.read_csv('CPR/CPR.csv')

,symbol,date,open,high,low,close,pivot,bc,tc,r1,s1,r2,s2,width,previous_high,previous_low,pivot_shifted,cpr_trend,day_open,cpr_opportunity
0,REEFUSDT,2022-06-07,0.004423,0.004471,0.004099,0.004256,0.004470,0.004494,0.004446,0.004599,0.004294,0.004775,0.004165,moderate,0.004646,0.004341,0.004383,uptrend,below,unclear
1,IOTXUSDT,2022-06-07,0.034570,0.034910,0.031730,0.033330,0.034760,0.034855,0.034665,0.035910,0.033420,0.037250,0.032270,moderate,0.036100,0.033610,0.034217,uptrend,below,unclear
2,XLMUSDT,2022-06-07,0.144380,0.145130,0.134210,0.138430,0.145670,0.146330,0.145010,0.149290,0.140730,0.154230,0.137110,moderate,0.150610,0.142050,0.144443,uptrend,below,unclear
3,OMGUSDT,2022-06-07,2.548000,2.576000,2.401000,2.481000,2.578667,2.593500,2.563833,2.691333,2.436333,2.833667,2.323667,moderate,2.721000,2.466000,2.463000,uptrend,below,unclear
4,BALUSDT,2022-06-07,7.511000,7.604000,6.825000,6.977000,7.618667,7.671500,7.565833,7.837333,7.294333,8.161667,7.075667,moderate,7.943000,7.400000,7.314000,uptrend,below,unclear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,API3USDT,2022-06-07,1.541000,1.570000,1.407000,1.430000,1.549333,1.553500,1.545167,1.599667,1.490667,1.658333,1.440333,moderate,1.608000,1.499000,1.508000,uptrend,below,unclear
138,TLMUSDT,2022-06-07,0.036100,0.036600,0.032300,0.033100,0.036067,0.036000,0.036133,0.037233,0.035033,0.038267,0.033867,moderate,0.037100,0.034900,0.035467,uptrend,Neutral,unclear
139,MATICUSDT,2022-06-07,0.628200,0.634900,0.581800,0.589800,0.626467,0.625600,0.627333,0.657733,0.596933,0.687267,0.565667,narrow,0.656000,0.595200,0.597233,uptrend,above,long
140,SNXUSDT,2022-06-07,2.719000,2.780000,2.502000,2.562000,2.738333,2.747500,2.729167,2.888667,2.569667,3.057333,2.419333,moderate,2.907000,2.588000,2.645667,uptrend,below,unclear


In [ ]:
pd.read_csv('CPR/CPR.csv')

,symbol,date,open,high,low,close,pivot,bc,tc,r1,s1,r2,s2,width,previous_high,previous_low,pivot_shifted,cpr_trend,day_open,cpr_opportunity
0,REEFUSDT,2022-06-07,0.004423,0.004471,0.004099,0.004256,0.004470,0.004494,0.004446,0.004599,0.004294,0.004775,0.004165,moderate,0.004646,0.004341,0.004383,uptrend,below,unclear
1,IOTXUSDT,2022-06-07,0.034570,0.034910,0.031730,0.033330,0.034760,0.034855,0.034665,0.035910,0.033420,0.037250,0.032270,moderate,0.036100,0.033610,0.034217,uptrend,below,unclear
2,XLMUSDT,2022-06-07,0.144380,0.145130,0.134210,0.138430,0.145670,0.146330,0.145010,0.149290,0.140730,0.154230,0.137110,moderate,0.150610,0.142050,0.144443,uptrend,below,unclear
3,OMGUSDT,2022-06-07,2.548000,2.576000,2.401000,2.481000,2.578667,2.593500,2.563833,2.691333,2.436333,2.833667,2.323667,moderate,2.721000,2.466000,2.463000,uptrend,below,unclear
4,BALUSDT,2022-06-07,7.511000,7.604000,6.825000,6.977000,7.618667,7.671500,7.565833,7.837333,7.294333,8.161667,7.075667,moderate,7.943000,7.400000,7.314000,uptrend,below,unclear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,API3USDT,2022-06-07,1.541000,1.570000,1.407000,1.430000,1.549333,1.553500,1.545167,1.599667,1.490667,1.658333,1.440333,moderate,1.608000,1.499000,1.508000,uptrend,below,unclear
138,TLMUSDT,2022-06-07,0.036100,0.036600,0.032300,0.033100,0.036067,0.036000,0.036133,0.037233,0.035033,0.038267,0.033867,moderate,0.037100,0.034900,0.035467,uptrend,Neutral,unclear
139,MATICUSDT,2022-06-07,0.628200,0.634900,0.581800,0.589800,0.626467,0.625600,0.627333,0.657733,0.596933,0.687267,0.565667,narrow,0.656000,0.595200,0.597233,uptrend,above,long
140,SNXUSDT,2022-06-07,2.719000,2.780000,2.502000,2.562000,2.738333,2.747500,2.729167,2.888667,2.569667,3.057333,2.419333,moderate,2.907000,2.588000,2.645667,uptrend,below,unclear


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b416ed7-4459-4831-a6c0-a85d5b5e69e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>